In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from joblib import dump, load
import matplotlib.pyplot as plt
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from data_loader import CustomSignalData, CustomSignalData1
from torch.autograd import Variable
from encoder import Encoder as E
from helpers import set_cmd_cb, rms_formuula, get_data, get_all_data, get_shift_data, get_operators, plot_cfs_mat, roll_data

DEVICE = torch.device("cpu")
def getFeatureMatrix(rawDataMatrix, windowLength, windowOverlap):
    rms = lambda sig: np.sqrt(np.mean(sig**2))
    nChannels,nSamples = rawDataMatrix.shape    
    I = int(np.floor(nSamples/(windowLength-windowOverlap)))
    featMatrix = np.zeros([nChannels, I])
    for channel in range(nChannels):
        for i in range (I):
            wdwStrtIdx=i*(windowLength-windowOverlap)
            sigWin = rawDataMatrix[channel][wdwStrtIdx:(wdwStrtIdx+windowLength-1)] 
            featMatrix[channel, i] = rms(sigWin)
    featMatrixData = np.array(featMatrix)
    return featMatrixData

class FFNN(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(FFNN, self).__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(inputSize, 9, bias=False),
            torch.nn.Sigmoid()
        )
        self.classifer = torch.nn.Sequential(
            torch.nn.Linear(9, outputSize, bias=False),
            # torch.nn.Softmax(dim=1)
        )

    def forward(self, x, encoder=None):
        if not encoder:
            encoder = self.encoder
        z = encoder(x)
        class_z = self.classifer(z)

        return class_z

class Operator(nn.Module):
    def __init__(self, in_features, n_rotations):
        super(Operator, self).__init__()
        """
        Args:
          in_features (int): Number of input features which should be equal to xsize.
          out_features (out): Number of output features which should be equal to ysize.
        """
        self.in_features = in_features
        self.core = torch.nn.Parameter(torch.zeros(3*self.in_features**2)- 0*torch.diag(torch.rand(3*self.in_features**2)/10))
        self.core.requires_grad = True
        self.n_rotations = n_rotations
        
    def rotate_batch(self, x, d, out_features):
      rotated = torch.empty(x.shape[0], 3*out_features*out_features, device=DEVICE)
      phies = [torch.linalg.matrix_power(self.core,i).to(DEVICE) for i in range (0,self.n_rotations+0)]
      for i in range (x.shape[0]):
        rotated[i] = phies[(d[i]+0)%4].matmul(x[i]) 
      return rotated

    def forward(self, x, d):
        """
        Args:
          x of shape (batch_size, 3, xsize, xsize): Inputs.
        
        Returns:
          y of shape (batch_size, 3*xsize^2): Outputs.
        """
        z = self.rotate_batch(x, d, self.in_features)
        return z
def get_tensor(arr):
    return torch.tensor(arr, device=DEVICE,dtype=torch.float )

def rotate_batch(x, d, out_features):
    M = torch.diag(torch.ones(8)).roll(-1,1)
    used_bases = [torch.linalg.matrix_power(M,i).to(DEVICE) for i in range (8)]
    rotated = torch.empty(x.shape, device=DEVICE)
    for i in range (x.shape[0]):
        rotated[i] = used_bases[d[i]].matmul(x[i]) 
    return rotated

def clf_acc(model, loader, masks = None, encoder = None):
    model.eval()
    correct = 0
    iter = 0
    with torch.no_grad():
        for inputs, labels,_,_ in loader:
            inputs = inputs.to(DEVICE)
            if masks is not None:
                inputs = inputs * masks[:inputs.size()[0]]
            labels = labels.to(DEVICE)
            labels = labels.flatten()
            if encoder:
                pred = model(inputs, encoder)
            else:
                pred = model(inputs)
            correct += (1-torch.abs(torch.sign(torch.argmax(pred,dim = 1)- labels))).mean().item()
            iter += 1
    return correct/iter

def compute_accuracy(a, b, loader):
    a.eval()
    b.eval()
    
    correct = 0
    iter = 0
    
    with torch.no_grad():
        for inputs1, inputs2, shift1, shift2, labels, _ in loader:
            inputs1 = inputs1.to(DEVICE)
            inputs2 = inputs2.to(DEVICE)
            shift1 = -shift1.int().flatten().to(DEVICE)
            shift2 = -shift2.int().flatten().to(DEVICE)
            labels = labels.flatten().to(DEVICE)
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            y1 = a(inputs1)
            y_tr_est1 = rotate_batch(y1,shift1,6)
            y_tr1 = b(y_tr_est1)

            y2 = a(inputs2)
            y_tr_est2 = rotate_batch(y2,shift1,6)
            y_tr2 = b(y_tr_est2)

            correct += (1-torch.abs(torch.sign(torch.argmax(y_tr1,dim = 1)- labels))).mean().item() + \
                    (1-torch.abs(torch.sign(torch.argmax(y_tr2,dim = 1)- labels))).mean().item()
            iter += 1
    return correct * 0.5 / iter

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
subject = '26'

Fs = 1000
windowLength = int(np.floor(0.1*Fs))  #160ms
windowOverlap =  int(np.floor(50/100 * windowLength))

# X_train = [np.zeros([0, 8]) for _ in range(9)]
# y_train = [np.zeros([0]) for _ in range(9)]
X_train = np.zeros([0,8])
y_train = np.zeros([0])
X_test = np.zeros([0,8])
y_test = np.zeros([0])



for shift in range(0,9):
    for files in sorted(os.listdir(f'Subject_{subject}/Shift_{shift}/')):
        _, class_,_, rep_ = files.split('_')
        
        df = pd.read_csv(f'Subject_{subject}/Shift_{shift}/{files}',skiprows=0,sep=' ',header=None)
        data_arr = np.stack([np.array(df.T[i::8]).T.flatten().astype('float32') for i in range (8)])
        data_arr -= 121
        data_arr /= 255.0
        feaData = getFeatureMatrix(data_arr, windowLength, windowOverlap)
        
        if not class_.startswith('9'):
            rms_feature = feaData.sum(0)
            baseline = 2*rms_feature[-50:].mean()
            start_ = np.argmax(rms_feature[::1]>baseline)
            end_  = -np.argmax(rms_feature[::-1]>baseline)
            feaData = feaData.T[start_:end_]
        else:
            feaData = feaData.T
        
        if shift == 0: 
            X_train = np.concatenate([X_train,feaData])
            y_train = np.concatenate([y_train,np.ones_like(feaData)[:,0]*int(class_)-1])
        
        else:
            X_test = np.concatenate([X_test,feaData])
            y_test = np.concatenate([y_test,np.ones_like(feaData)[:,0]*int(class_)-1])


In [4]:
print("Train dataset feature:")
print(X_train.shape)
print("Train dataset label:")
print(y_train.shape)
print("Testing dataset feature:")
print(X_test.shape)
print("Testing dataset label:")
print(y_test.shape)

Train dataset feature:
(7108, 8)
Train dataset label:
(7108,)
Testing dataset feature:
(55862, 8)
Testing dataset label:
(55862,)


In [6]:




all_X_train, all_y_train, all_shift_train = get_all_data(X_train, y_train)
all_X1_train, all_X2_train, all_shift_1_train, all_shift_2_train, all_y_shift_train = get_shift_data(all_X_train, all_shift_train, all_y_train)

triplet_train_dataset = CustomSignalData1(get_tensor(all_X1_train), get_tensor(all_X2_train), get_tensor(all_shift_1_train), get_tensor(all_shift_2_train), get_tensor(all_y_shift_train))
triplettrainloader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size = 102, shuffle=True)


In [8]:
all_X_train, all_y_train, all_shift_train = get_all_data(X_train, y_train)
encoder = E(8,8)
encoder.to(DEVICE)
classifier = FFNN(8,9)
classifier.to(DEVICE)

parameters = list(encoder.parameters()) + list(classifier.parameters())

crit1 = torch.nn.MSELoss()
crit2 = torch.nn.CrossEntropyLoss()
crit1.to(DEVICE)
crit2.to(DEVICE)
loss_record = []

optimizer = torch.optim.Adam(parameters, lr=0.002)
n_epochs = 50

for epoch in range(0,n_epochs):
    encoder.train()
    classifier.train()
    for inputs1, inputs2, shift1, shift2, labels, _ in triplettrainloader:
        inputs1 = inputs1.to(DEVICE)
        inputs2 = inputs2.to(DEVICE)
        shift1 = -shift1.int().flatten().to(DEVICE)
        shift2 = -shift2.int().flatten().to(DEVICE)
        labels = labels.long().flatten().to(DEVICE)
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        y1 = encoder(inputs1)
        y_tr_est1 = rotate_batch(y1,shift1,6)
        y_tr1 = classifier(y_tr_est1)


        y2 = encoder(inputs2)
        y_tr_est2 = rotate_batch(y2,shift2,6)
        y_tr2 = classifier(y_tr_est2)

        loss =  crit1(y_tr_est1, y_tr_est2) + 0.5*crit2(y_tr1,labels)+ 0.5*crit2(y_tr2,labels)
        loss.backward()
        optimizer.step()

torch.save(classifier.state_dict(), f"classifier_shift_0_s2_1.pt")
torch.save(encoder.state_dict(), f"encoder_shift_0_s2_1.pt")
with torch.no_grad():
    encoder.eval()
    N_points = 1000
    rand_idx = np.random.choice(all_X_train.shape[0], N_points)
    y_tr = encoder(get_tensor(all_X_train[rand_idx]))
    recovered_points_ = rotate_batch(y_tr,-all_shift_train[rand_idx].flatten(), 6)
    del y_tr

torch.save(recovered_points_, f"reference_points_shift_0_s2_1.pt")

In [9]:
DEVICE = torch.device("cpu") # operations is in CPU or GPU.
M = torch.diag(torch.ones(8)).roll(-1,1) # Create a diagnoise matrix then shift it to the right
used_bases = [torch.linalg.matrix_power(M,i).to(DEVICE) for i in range (8)] #

N_points = 1000



classifier = FFNN(8,9) # This indicates that the neural network expects input data with 8 features and will produce output predictions across 9 classes.
encoder = E(8,8)
encoder.load_state_dict(torch.load(f"encoder_shift_0_s2_1.pt")) # contains the learned parameters (weights and biases) of the encoder model
recovered_points_= torch.load(f"reference_points_shift_0_s2_1.pt") # These points represent reference points for inference or evaluation in the model
classifier.load_state_dict(torch.load(f"classifier_shift_0_s2_1.pt")) # contains the weights and biases learned during training.
classifier.eval() # sets the model to evaluation mode.
encoder.eval() # sets the model to evaluation mode.

# Get the test data for the current shift and fold
X_test_shift = X_test
# Encode the test data using the encoder model
y1 = encoder(get_tensor(X_test_shift))

# Use the encoded data directly without rotation
y_tr1 = classifier(y1).argmax(1)

# Calculate accuracy and append to list
accuracies_self_supervised_list_s2 = (1 - torch.abs(torch.sign(y_tr1 - get_tensor(y_test).flatten()))).mean().item()
print(accuracies_self_supervised_list_s2)

0.19388851523399353
